<a href="https://colab.research.google.com/github/AnnSenina/Python_CL_2023/blob/main/homework/Hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 2

## pandas + парсинг

Здесь находится главная страница [Системного блока](https://sysblok.ru)

Соберите корпус новостей (можно спарсить все новости или небольшую часть)

Дедлайн: 31 декабря включительно

In [5]:
# необходимые импорты
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd

import time #добавим дополнительный импорт - для работы со временем (чтобы сервер не ругался)

### шаг 1

In [8]:
all_pages = ['https://sysblok.ru', ]

# Вариант кода, если заранее известно, сколько страниц надо перебрать

#for page_num in range(2, 17):
#  all_pages.append(f"https://sysblok.ru/page/{page_num}/")

# Если заранее неизвестно, сколько будет страниц, то можно заполнить список с использованием цикла while

page_num = 2
while True:
  url = f"https://sysblok.ru/page/{page_num}/"
  page = rq.get(url)
  if page.status_code == 404:
    break
  else:
    all_pages.append(url)
  page_num += 1
  time.sleep(2)

print(all_pages)

['https://sysblok.ru', 'https://sysblok.ru/page/2/', 'https://sysblok.ru/page/3/', 'https://sysblok.ru/page/4/', 'https://sysblok.ru/page/5/', 'https://sysblok.ru/page/6/', 'https://sysblok.ru/page/7/', 'https://sysblok.ru/page/8/', 'https://sysblok.ru/page/9/', 'https://sysblok.ru/page/10/', 'https://sysblok.ru/page/11/', 'https://sysblok.ru/page/12/', 'https://sysblok.ru/page/13/', 'https://sysblok.ru/page/14/', 'https://sysblok.ru/page/15/', 'https://sysblok.ru/page/16/']


### шаг 2

In [9]:
# Cобираем ссылки на отдельные новости с каждой страницы

all_links = []

for url in all_pages:
  page = rq.get(url)
  soup = bs(page.text, features="html.parser")
  for anchor in soup.find_all("a"):
    if anchor.parent.name == "h2" and anchor.parent.parent.get("class")[0] == "article":
      all_links.append(anchor.get("href"))
  time.sleep(2)

print(all_links)


['https://sysblok.ru/visual/recept-nobelevskoj-premii-issleduem-otkrytye-dannye-o-laureatah/', 'https://sysblok.ru/urban/kak-cifrovye-metody-spasajut-ljudej-ot-avarij/', 'https://sysblok.ru/test/krash-test-dlja-ljubitelej-deshjovyh-dram/', 'https://sysblok.ru/test/iz-rossii-s-dannymi-uznajte-region-po-statistike/', 'https://sysblok.ru/oriental/ot-korpusov-do-karty-bombardirovok-hirosimy-digital-humanities-v-japonii/', 'https://sysblok.ru/philology/zhizn-stala-chjorno-beloj-kak-menjalsja-jazyk-pojezii-silvii-plat/', 'https://sysblok.ru/arts/otmyt-istoriju-sankt-peterburga-kak-jentuziasty-vosstanavlivajut-pamjatniki/', 'https://sysblok.ru/technohistory/roboty-v-filmah-xx-veka-ot-nenavisti-k-ii-do-slijanija/', 'https://sysblok.ru/society/slovo-pacana-kak-internet-fenomen-chto-ishhut-v-poiskovikah-zriteli-seriala/', 'https://sysblok.ru/urban/segodnja-zavod-zavtra-futbolnoe-pole-kak-po-snimkam-iz-kosmosa-izuchajut-jevoljuciju-goroda/', 'https://sysblok.ru/oriental/ljudi-na-ladoni-biografich

### шаг 3

работаем с 1 новостью:

нас интересует название, автор, дата публикации, текст (дополнительно можно спарсить тематические категории)

In [11]:
# Парсинг даты: извлечем текст из тега time
# Создаем функцию для получения отдельной новости по URL

def get_post(url):
  post_page = rq.get(url)
  post = bs(post_page.text, features="html.parser")
  return post

# Создаем функцию, чтобы получить информацию о дате и времени публикации отдельной новости
# На странице новости может быть несколько тегов time, но только один из них нам необходимо извлечь
# (в данном случае -- тот, который вложен в тег div с атрибутом class, где первый элемент post-meta-and-title)

def get_post_datetime(post):
  post_time = ""
  for time in post.find_all("time"):
    if time.parent.parent.get("class")[0] == "post-meta-and-title":
      post_time = time.text
  return post_time

soup = get_post(all_links[0])
soup_datetime = get_post_datetime(soup)

print(soup_datetime)

29.12.2023


In [12]:
# Парсинг заголовка: извлечем текст из тега h1
# Создаем функцию для извлечения заголовка статьи
def get_post_title(post):
  return post.find("h1").text

post_title = get_post_title(soup)
print(post_title)

Рецепт Нобелевской премии: исследуем открытые данные о лауреатах


In [13]:
# Парсинг текста: соберем все тексты из тега p и соединим в строку
# Создаем функцию для извлечения основного текста статьи
# Исходим из того, что все теги p вложены в тег article,
# что родительским элементом для тега p не может быть тег div (метаданные внизу статьи),
# а также из того, что дочерним элементом для тега p не может быть тег em (подписи к рисункам)
def get_post_text(post):
  post_parts = []
  for p in list(filter(lambda x: x.parent.name != "div" and x.find("em", recursive=False) == None, post.find("article").find_all("p"))):
      post_parts.append(p.text)
  return "\n".join(post_parts)

print(get_post_text(soup))

Несколько фактов о Нобелевской премии:
Нобелевский комитет поддерживает замечательный сайт, на котором собраны сведения обо всех лауреатах и награждениях. С помощью API можно скачать данные за период с 1901 по 2023 год и исследовать любопытные закономерности.
Изучив данные о рождении и смерти, можно выяснить, сколько в среднем живёт нобелевский лауреат.
Как видно, средняя продолжительность жизни 820 уже умерших лауреатов составляет около 81 года. При этом пятеро лауреатов прожили более ста лет (среди них — Генри Киссинджер, скончавшийся 29 ноября 2023 года в возрасте 100 лет). Два лауреата здравствуют и поныне: 101-летний физик Янг Чжэньнин (премия 1957 года) и 100-летний химик Рудольф Маркус (премия 1992 года). Абсолютный рекордсмен среди нобелевских долгожителей — итальянка Рита Леви-Монтальчини, лауреат премии по медицине 1986 года, прожившая 103 года. Раньше всех умер Мартин Лютер Кинг, лауреат премии мира. Он был убит в возрасте 39 лет.
Нобелевскую премию вручают уже более ста лет

In [14]:
# Извлекаем информацию об авторе статьи из тега  meta
# Создаем функцию для извлечения информации об авторе для отдельной новости
# На всякий случай исходим из того, что таких тегов может быть более одного
# (хотя в тех статьях, где два автора, вроде бы они перечислены через запятую, а тег один)

def get_post_authors(post):
  author_list = []
  for meta in list(filter(lambda x: x.get("name") == "author", post.find_all("meta"))):
      author_list.append(meta.get("content"))
  return ", ".join(author_list)

print(get_post_authors(soup))

Евгений Дуненков


In [15]:
# Извлекаем информацию о тематических категориях:
# Создаем для этого функцию.

def get_post_categories(post):
  categories = []
  for a in list(filter(lambda a: a.get("rel") == ["category", "tag"], post.find_all("a"))):
    categories.append(a.text)
  return ", ".join(categories)

print(get_post_categories(soup))

Визуализация, Общество


### шаг 4

In [16]:
# Используя созданные на предыдущих этапах функции создаем общую функцию со своей необходимой информацией для отдельной новости
# Информация представлена в виде списка

def get_post_info(post):
  info = []
  info.extend([get_post_title(post), get_post_authors(post), get_post_datetime(post), get_post_categories(post), get_post_text(post)])
  return info

print(get_post_info(soup))

['Рецепт Нобелевской премии: исследуем открытые данные о лауреатах', 'Евгений Дуненков', '29.12.2023', 'Визуализация, Общество', 'Несколько фактов о Нобелевской премии:\nНобелевский комитет поддерживает замечательный сайт, на котором собраны сведения обо всех лауреатах и награждениях. С помощью API можно скачать данные за период с 1901 по 2023 год и исследовать любопытные закономерности.\nИзучив данные о рождении и смерти, можно выяснить, сколько в среднем живёт нобелевский лауреат.\nКак видно, средняя продолжительность жизни 820 уже умерших лауреатов составляет около 81 года. При этом пятеро лауреатов прожили более ста лет (среди них — Генри Киссинджер, скончавшийся 29 ноября 2023 года в возрасте 100 лет). Два лауреата здравствуют и поныне: 101-летний физик Янг Чжэньнин (премия 1957 года) и 100-летний химик Рудольф Маркус (премия 1992 года). Абсолютный рекордсмен среди нобелевских долгожителей — итальянка Рита Леви-Монтальчини, лауреат премии по медицине 1986 года, прожившая 103 года.

In [18]:
# Импортируем random, чтобы сет новостей формировался произвольно, в нашем случае желаемый объем выборки = 20 постов

import random

total = 20
short_list = random.sample(all_links, total)

posts_in_list = []

for url in short_list:
  post = get_post(url)
  posts_in_list.append(get_post_info(post))

print(short_list)
print(posts_in_list)

['https://sysblok.ru/metascience/kak-ne-stat-zhertvoj-fejk-njuc-sovety-uchenyh/', 'https://sysblok.ru/society/nauchim-ljubit-rodinu-i-mat-tvoju-o-chjom-govorjat-na-razgovorah-o-vazhnom/', 'https://sysblok.ru/musicology/nichego-svjatogo-matematiki-otdelili-makkartni-ot-lennona/', 'https://sysblok.ru/society/ja-my-nkrja-chto-proishodit-s-nacionalnym-korpusom/', 'https://sysblok.ru/nlp/kak-mashinnyj-perevod-ocenivaet-mashina/', 'https://sysblok.ru/philology/shaherezada-robot-rasskazyvajushhij-istorii/', 'https://sysblok.ru/visual/data-storitelling-dannye-govorjat-za-sebja/', 'https://sysblok.ru/arts/cifrovaja-rekonstrukcija-kak-vossozdali-portret-cherchillja-dlja-seriala-korona/', 'https://sysblok.ru/nlp/nejroset-nauchilas-diagnostirovat-depressiju/', 'https://sysblok.ru/society/kak-ustroeno-obshhenie-iskusstvennogo-intellekta-s-potencialnymi-kandidatami-i-pochemu-nejroset-tozhe-mozhet-diskriminirovat/', 'https://sysblok.ru/society/otdaj-rabotu-robotu/', 'https://sysblok.ru/neuroscience/n

### шаг 5

In [23]:
# Cоберем всю информацию из списка в датафрейм
df = pd.DataFrame(posts_in_list, columns=["Title", "Authors", "Date", "Categories", "Text"])

df

,Title,Authors,Date,Categories,Text
0,Как не стать жертвой фейк ньюc: советы ученых,Ana Coughlin,24.10.2020,"Науковедение, Общество","Все мы слышали, что с приходом цифровых технол..."
1,Научим любить Родину… и мать твою: о чём говор...,Анна Оськина,06.10.2023,"Образование, Общество",В первый год существования внеклассные уроки «...
2,Ничего святого: математики отделили Маккартни ...,Нелли Бурцева,23.01.2019,Музыка,"Общеизвестно, что почти до последнего альбома ..."
3,Я/МЫ НКРЯ: что происходит с национальным корпусом,Системный Блокъ,07.10.2019,Общество,8 сентября ряд функций Национального корпуса р...
4,Как машинный перевод оценивает… машина?,Системный Блокъ,30.04.2019,NLP,"Оценивая качество машинного перевода (МП), чел..."
5,"«Шахерезада»: робот, рассказывающий истории",Кристина Сопрано,31.01.2020,Филология,"Большинство ИИ, умеющих писать художественную ..."
6,Data-сторителлинг: данные говорят за себя,Денис Колобов,20.12.2018,Визуализация,"Сторителлинг — это рассказ качественных, запом..."
7,Цифровая реконструкция: как воссоздали портрет...,Полина Забадыкина,19.05.2023,Искусство,Мастерство художников и современные технологии...
8,Нейросеть научилась диагностировать депрессию,Даниил Скоринкин,06.10.2018,NLP,Исследователи из Массачуссетского технологичес...
9,Алгоритмы и дискриминация на рынке труда. Чему...,Юлия Токарева,04.12.2021,Общество,"Mercer, крупнейшая в мире консалтинговая фирма..."


In [22]:
df.to_excel("20posts.xlsx")

Готово!
Загрузите ваш код и собранные новости на GitHub